<a href="https://colab.research.google.com/github/Hm552/signate-systematic-review-246th/blob/main/keras_lgb_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# データセット保存先ディレクトリ
if 'google.colab' in sys.modules:
    dataset_dir="/content/drive/MyDrive/SRWS"
else:
    dataset_dir="/path/to"

In [ ]:
! pip install --quiet transformers

     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 636 kB 73.4 MB/s 
     |████████████████████████████████| 895 kB 71.2 MB/s 
     |████████████████████████████████| 3.3 MB 66.6 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 


In [ ]:
! pip install --quiet tensorflow-addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [ ]:
import logging
import datetime
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import fbeta_score
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from scipy.optimize import minimize, minimize_scalar

from tensorflow.keras import backend as K
import tensorflow as tf
#import tensorflow_addons as tfa

#import transformers
#from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification

import lightgbm as lgb
from sklearn.svm import SVC

In [ ]:
class Logger:
    """log を残す用のクラス"""
    def __init__(self, path):
        self.general_logger = logging.getLogger(path)
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler(os.path.join(path, 'Experiment.log'))
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)

    def info(self, message):
        # display time
        self.general_logger.info('[{}] - {}'.format(self.now_string(), message))

    @staticmethod
    def now_string():
        return str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
import os
INPUT = os.path.join(dataset_dir, "input")
OUTPUT = os.path.join(dataset_dir, "output")
EXP = "211002_1"
OUTPUT_EXP = os.path.join(OUTPUT, EXP)
OUTPUT_EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
OUTPUT_EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")
OUTPUT_EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
SUBMISSION = os.path.join(dataset_dir, "submission")

In [ ]:
warnings.filterwarnings("ignore")  # ignore warnings
logger = Logger(OUTPUT_EXP)  # set logger 

# TPU setting
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', TPU.cluster_spec().as_dict()['worker'])
except ValueError:
    TPU = None
    print('INFO: Not connected to a TPU runtime')

Running on TPU  ['10.27.242.2:8470']


In [ ]:
def build_auto_model():
    """TFAutoModel"""
    transformer = TFAutoModel.from_pretrained(Config.model)
    input_word_ids = tf.keras.layers.Input(shape=(Config.max_length, ), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(Config.max_length, ), dtype=tf.int32, name='attention_mask')

    x = transformer(input_word_ids, attention_mask=attention_mask)
    #x = x[0][:, 0, :]  # cls

    last_hidden_state = x[0]
    last_hidden_state = tf.keras.layers.Dropout(0.1)(last_hidden_state)
    x_avg = tf.keras.layers.GlobalAveragePooling1D()(last_hidden_state)
    x_max = tf.keras.layers.GlobalMaxPooling1D()(last_hidden_state)
    x = tf.keras.layers.Concatenate()([x_avg, x_max])

    samples = []    
    for n in range(8):
        sample_mask = tf.keras.layers.Dense(64, activation='relu', name = f'dense_{n}')
        sample = tf.keras.layers.Dropout(.5)(x)
        sample = sample_mask(sample)
        sample = tf.keras.layers.Dense(1, activation='sigmoid', name=f'sample_{n}')(sample)
        samples.append(sample)
      
    output = tf.keras.layers.Average(name='output')(samples)

    #output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=[input_word_ids, attention_mask],
                                  outputs=[output])

    optimizer = tfa.optimizers.AdamW(lr=Config.lr, weight_decay=Config.weight_decay)
    model.compile(optimizer = optimizer,
                  loss = [tf.keras.losses.BinaryCrossentropy()],  
                  metrics = ["accuracy"]) 
    return model

In [ ]:
class Config:
    name_v1 = "tf-roberta-base"
    model = "roberta-base"
    max_length = 256
    lr = 0.00002
    weight_decay = 1e-5
    n_fold = 4 
    epochs = 32
    train_batch_size = 64
    steps_per_epochs = 30
    seeds = [2021]
    target_col = "judgement"
    text_col = "text"  # title + abst
    valid_batch_size = 64
    test_batch_size = 64
    class_weight = None
    submit = True
    debug = False

if Config.debug:
    Config.epochs = 1
    Config.n_fold = 2

In [ ]:
def opt_fbeta_threshold(y_true, y_pred):
    """fbeta score計算時のthresholdを最適化"""
    def opt_(x): 
        return -fbeta_score(y_true, y_pred >= x, beta=7)
    result = minimize(opt_, x0=np.array([0.1]), method='Powell')
    best_threshold = result['x'].item()
    return best_threshold


def metrics(y_true, y_pred):
    """fbeta(beta=7)の閾値最適化評価関数"""
    bt = opt_fbeta_threshold(y_true, y_pred)
    print(f"bt:{bt}")
    score = fbeta_score(y_true, y_pred >= bt, beta=7)
    return score


def skf(train, n_splits, random_state):
    """層化KFoldのインデックスのリストを作成"""
    skf = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    return list(skf.split(train, train[Config.target_col]))


def encode_texts(texts, tokenizer, max_length):
    """text encoding"""
    encoded_dict = tokenizer.batch_encode_plus(
        texts,
        padding = 'max_length',
        truncation = True,
        max_length = max_length,
    )
    return dict(encoded_dict)

def get_dataset(X, y=None, dataset="test"):
    """データをtf.Datasetの形式に変更"""
    if dataset=="train":
        train_dataset = (
            tf.data.Dataset
            .from_tensor_slices((X, y))
            .shuffle(2048)
            .batch(Config.train_batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
            )
        if Config.steps_per_epochs is not None:
            train_dataset = train_dataset.repeat()

        return train_dataset

    elif dataset=="valid":
        valid_dataset = (
            tf.data.Dataset
            .from_tensor_slices((X, y))
            .batch(Config.valid_batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
        return valid_dataset
    
    elif dataset=="test":
        test_dataset = (
            tf.data.Dataset
            .from_tensor_slices(X)
            .batch(Config.test_batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
        return test_dataset


def get_model_and_tokenizer():
    """model&tokenizer を取得 (TPU ok)"""
    if TPU:
        tf.config.experimental_connect_to_cluster(TPU)
        tf.tpu.experimental.initialize_tpu_system(TPU)
        tpu_strategy = tf.distribute.experimental.TPUStrategy(TPU)
        with tpu_strategy.scope():
            model = build_auto_model()
    else:
        model = build_auto_model()

    tokenizer = AutoTokenizer.from_pretrained(Config.model)
    return model, tokenizer


def get_class_weight(target, weight):
    """class weights の作成"""
    if weight == "balanced":
        class_weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(target),
            y=target)
        class_weights = dict(enumerate(class_weights))
    elif weight is None:
        class_weights = None
    else:
        # ex) weight = {0:0.2, 1:0.98}
        class_weights = weight
    
    return class_weights


def training_v1(train_df, valid_df, model, tokenizer, filepath):
    """training 用の関数"""
    tr_text = encode_texts(texts=train_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    va_text = encode_texts(texts=valid_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    tr_dataset = get_dataset(X=tr_text, y=train_df[Config.target_col].values, dataset="train")
    va_dataset = get_dataset(X=va_text, y=valid_df[Config.target_col].values, dataset="valid")

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath, 
        moniter="val_loss", 
        verbose=1, 
        save_best_only=True, 
        save_weights_only=True,
        mode="min")
    
    model.fit(tr_dataset, 
              epochs=Config.epochs, 
              verbose=1, 
              callbacks=[checkpoint],
              validation_data=va_dataset, 
              steps_per_epoch=Config.steps_per_epochs,
              class_weight=get_class_weight(train_df[Config.target_col], weight=Config.class_weight))


def inference_v1(test_df, model, tokenizer, filepath):
    """推論用の関数"""
    model.load_weights(filepath)
    te_text = encode_texts(texts=test_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    te_dataset = get_dataset(X=te_text, y=None, dataset="test")
    preds = model.predict(te_dataset)


    return preds.reshape(-1)

'''def greed_encode(data, max_len) :
    input_ids = []
    attention_masks = []
  
    for i in range(len(data.text)):
        
        encoded = tokenizer.encode_plus(data.text[i], add_special_tokens=True, max_length=max_len, pad_to_max_length=True)
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids),np.array(attention_masks)'''

def features_from_bertlayer(train_df, test_df, model, tokenizer, filepath, name):
    """推論用の関数"""
    #model.load_weights(filepath)
    cls_layer_model = tf.keras.models.Model(model.input, outputs=model.get_layer(f'dense_1').output)

    tr_text = encode_texts(texts=train_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    #tr_dataset = get_dataset(X=tr_text, y=train_df[Config.target_col].values, dataset="train")
    te_text = encode_texts(texts=test_df[Config.text_col].tolist(), tokenizer=tokenizer, max_length=Config.max_length)
    #te_dataset = get_dataset(X=te_text, y=None, dataset="test")

    X_train = cls_layer_model.predict([np.array(tr_text['input_ids']), np.array(tr_text['attention_mask'])])
    X_test = cls_layer_model.predict([np.array(te_text['input_ids']), np.array(te_text['attention_mask'])])
    y_train = train_df[Config.target_col].values

    return X_train, X_test, y_train

def lgb_cv(X_train, X_test, y_train, name):
    N_FOLDS = 3
    print(f'LGBM')
    folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    oof = np.zeros(len(X_train))
    sub = np.zeros(len(X_test))
    params = {'boosting_type': 'dart'}
    for fold_, (train_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        X_train_cv, y_train_cv = pd.DataFrame(X_train).loc[train_idx], pd.DataFrame(y_train).loc[train_idx]
        X_val, y_val = pd.DataFrame(X_train).loc[val_idx], pd.DataFrame(y_train).loc[val_idx]
        train_data = lgb.Dataset(X_train_cv, label=y_train_cv)
        val_data = lgb.Dataset(X_val, label=y_val)
        watchlist = [train_data, val_data]
        clf = lgb.train(params, train_set = train_data, valid_sets=watchlist)
        oof[val_idx] = clf.predict(X_val)
        sub += clf.predict(X_test)/folds.n_splits
    oof_all_lgb = oof
    sub_all_lgb = sub
    #print(accuracy_score(y_train, np.round(oof).astype(int)),'\n')
    score = metrics(y_train, oof_all_lgb)
    logger.info(f"{name}_lgb >>> val score:{score:.4f}")
    return oof_all_lgb, sub_all_lgb

def svc_cv(X_train, X_test, y_train, name):
    N_FOLDS = 3
    print(f'SVC')
    folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    oof = np.zeros(len(X_train))
    sub = np.zeros(len(X_test))
    scores = [0 for _ in range(folds.n_splits)]
    for fold_, (train_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
        X_train_cv, y_train_cv = pd.DataFrame(X_train).loc[train_idx], pd.DataFrame(y_train).loc[train_idx]
        X_val, y_val = pd.DataFrame(X_train).loc[val_idx], pd.DataFrame(y_train).loc[val_idx]
        clf = SVC(kernel='rbf', C=1.75, gamma = 0.1, probability = True).fit(X_train_cv, y_train_cv)
        oof[val_idx] = clf.predict_proba(X_val)[:,1]
        sub += clf.predict_proba(X_test)[:,1]/folds.n_splits
    oof_all_svc = oof
    sub_all_svc = sub    
    #print(accuracy_score(y_train, np.round(oof).astype(int)),'\n')
    score = metrics(y_train, oof_all_svc)
    logger.info(f"{name}_svc >>> val score:{score:.4f}")
    return oof_all_svc, sub_all_svc


def train_cv_v1(train, cv, metrics, name, dir):
    """cross validationの実行関数 (train)"""

    oof = np.zeros(len(train))
    for i_fold, (tr_idx, va_idx) in enumerate(cv):
        K.clear_session()
        model, tokenizer = get_model_and_tokenizer()

        tr_df, va_df = train.iloc[tr_idx].reset_index(), train.iloc[va_idx].reset_index()
        filepath = os.path.join(dir, f"{name}_fold{i_fold+1}.h5")

        if not os.path.isfile(filepath):  # 学習済みモデルがあればtrainingしない
            training_v1(tr_df, va_df, model, tokenizer, filepath)
        
        preds = inference_v1(va_df, model, tokenizer, filepath)

        score = metrics(np.array(va_df[Config.target_col]), preds)
        logger.info(f"{name}_fold{i_fold+1} >>> val socre:{score:.4f}")
        oof[va_idx] = preds
    
    score = metrics(np.array(train[Config.target_col]), oof)
    logger.info(f"{name} >>> val score:{score:.4f}")
    return oof
        
def predict_cv_lgb_svc(train, test, name, dir):
    """cross validationの実行関数 (test)"""
    lgb_preds_fold = []
    svc_preds_fold = []
    lgb_oof_fold = []
    svc_oof_fold = []
    for i_fold in range(Config.n_fold):
        model, tokenizer = get_model_and_tokenizer()
        filepath = os.path.join(dir, f"{name}_fold{i_fold+1}.h5")
        # grab last layer and use LGBM and SVC
        X_train, X_test, y_train = features_from_bertlayer(train, test, model, tokenizer, filepath, name)
        lgb_oof, lgb_preds = lgb_cv(X_train, X_test, y_train, name)
        svc_oof, svc_preds = svc_cv(X_train, X_test, y_train, name)

        lgb_preds_fold.append(lgb_preds)
        svc_preds_fold.append(svc_preds)
        lgb_oof_fold.append(lgb_oof)
        svc_oof_fold.append(svc_oof)

        logger.info(f"{name}_fold{i_fold+1} inference")
    
    lgb_pred = np.mean(lgb_preds_fold, axis=0)
    svc_pred = np.mean(svc_preds_fold, axis=0)
    lgb_oof_ = np.mean(lgb_oof_fold, axis=0)
    svc_oof_ = np.mean(svc_oof_fold, axis=0)
    return lgb_pred, svc_pred, lgb_oof_, svc_oof_

def predict_cv_v1(train, test, name, dir):
    """cross validationの実行関数 (test)"""
    preds_fold = []
    for i_fold in range(Config.n_fold):
        model, tokenizer = get_model_and_tokenizer()
        filepath = os.path.join(dir, f"{name}_fold{i_fold+1}.h5")
        preds = inference_v1(test, model, tokenizer, filepath)
        preds_fold.append(preds)

        logger.info(f"{name}_fold{i_fold+1} inference")
    
    pred = np.mean(preds_fold, axis=0)
    return pred

def visualize_confusion_matrix(
        y_true,
        pred_label,
        height=.6,
        labels=None):
    """混合行列をプロット 
    (https://www.guruguru.science/competitions/11/discussions/2fb11851-67d0-4e96-a4b1-5629b944f363/)"""
    
    conf = confusion_matrix(y_true=y_true,
                            y_pred=pred_label,
                            normalize='true')

    n_labels = len(conf)
    size = n_labels * height
    fig, ax = plt.subplots(figsize=(size * 4, size * 3))
    sns.heatmap(conf, cmap='Blues', ax=ax, annot=True, fmt='.2f')
    ax.set_ylabel('Label')
    ax.set_xlabel('Predict')

    if labels is not None:
        ax.set_yticklabels(labels)
        ax.set_xticklabels(labels)
        ax.tick_params('y', labelrotation=0)
        ax.tick_params('x', labelrotation=90)

    plt.show()
    return fig

In [ ]:
def main():
    # load data
    train = pd.read_csv(os.path.join(INPUT, "train.csv"))
    test = pd.read_csv(os.path.join(INPUT, "test.csv"))

    if Config.debug:
        train = train.sample(1000, random_state=Config.seeds[0]).reset_index(drop=True)
        test = test.sample(1000, random_state=Config.seeds[0]).reset_index(drop=True)

    # preprocess
    train["text"] = train["title"] + " " + train["abstract"].fillna("")
    test["text"] = test["title"] + " " + test["abstract"].fillna("")

    # training
    oof_df = pd.DataFrame()
    for seed in Config.seeds:
        name = f"{Config.name_v1}-{seed}"
        oof = train_cv_v1(train, 
                        cv=skf(train, n_splits=Config.n_fold, random_state=seed),
                        metrics=metrics, 
                        name=name, 
                        dir=OUTPUT_EXP_MODEL)
        oof_df[name] = oof
    oof_df.to_csv(os.path.join(OUTPUT_EXP_PREDS, "tf_oof.csv"), index=False)  # oof 予測値(prob)を保存

    #oof_df = pd.read_csv(os.path.join(OUTPUT_EXP_PREDS, "tf_oof.csv"))  # oof 予測値(prob)を保存

    # inference
    preds_df = pd.DataFrame()
    for seed in Config.seeds:
        name = f"{Config.name_v1}-{seed}"
        pred = predict_cv_v1(train, test, name, dir=OUTPUT_EXP_MODEL)
        preds_df[name] = pred
    
    preds_df.to_csv(os.path.join(OUTPUT_EXP_PREDS, "tf_preds.csv"), index=False)  # test予測値(prob)を保存
    #preds_df = pd.read_csv(os.path.join(OUTPUT_EXP_PREDS, "tf_oof.csv"))  # oof 予測値(prob)を保存
    
    # inference
    for seed in Config.seeds:
        name = f"{Config.name_v1}-{seed}"
        lgb_pred, svc_pred, lgb_oof_, svc_oof_ = predict_cv_lgb_svc(train, test, name, dir=OUTPUT_EXP_MODEL)
        preds_df[name + "_lgb"] = lgb_pred
        preds_df[name + "_svc"] = svc_pred
        oof_df[name + "_lgb"] = lgb_oof_
        oof_df[name + "_svc"] = svc_oof_
    
    oof_df.to_csv(os.path.join(OUTPUT_EXP_PREDS, "oof.csv"), index=False)  # oof 予測値(prob)
    preds_df.to_csv(os.path.join(OUTPUT_EXP_PREDS, "preds.csv"), index=False)  # test予測値(prob)を保存

    y_true = train[Config.target_col]
    y_pred = oof_df.mean(axis=1)

    best_threshold = opt_fbeta_threshold(y_true.values, y_pred.values)
    oof_score = fbeta_score(y_true, y_pred >= best_threshold, beta=7)
    comments = f"score:{oof_score:.4f}/threshold:{best_threshold}"  # 最終的なスコアと閾値 (スペースがあるとsubmit時のコメントエラーが出る)
    logger.info(comments)

    fig = visualize_confusion_matrix(y_true, y_pred>=best_threshold)  # 混合行列を表示&save
    fig.savefig(os.path.join(OUTPUT_EXP_FIG, "cm.png"), dpi=300)

    # submit
    if Config.submit:
        sub_df = pd.read_csv(os.path.join(INPUT, "sample_submit.csv"), header=None)
        sub_df.columns = ["id", "judgement"]

        preds = preds_df.mean(axis=1)
        sub_df["judgement"] = (preds.values >= best_threshold) * 1
        
        filepath = os.path.join(SUBMISSION, f"{EXP}.csv")
        sub_df.to_csv(filepath, index=False, header=False)
        
        # signate api を使ってsubmit
        #! signate submit --competition-id=471 $filepath --note $comments

In [ ]:
if __name__ == "__main__":
    main()